# Opdracht satellietdata analyseren
In deze opdracht ga je met échte satellietdata werken om de temperatuurverandering in Bonaire te analyseren. Je gaat met de volgende dataset werken:

Muñoz Sabater, J. (2019): ERA5-Land monthly averaged data from 1950 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). DOI: 10.24381/cds.68d2bb30 (Accessed on DD-MMM-YYYY)

---
Als eerst moeten we de nodige Python-libraries importeren.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

YOUR_CODE_HERE = None

Dan openen we de dataset.

In [ ]:
ds = xr.open_dataset('data.grib').metpy.parse_cf()
ds

De temperatuur wordt in `Kelvin` gegeven. Wat moeten we doen met de data om dit naar `Celsius` te veranderen? 

In [ ]:
ds = ds - YOUR_CODE_HERE 

Laten we een kijkje nemen naar de data. Als je de onderstaande code runt, zal je een kaart zien van Carribisch Nederland.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(resolution="10m")
plot = ds.stl1[0].plot(
    cmap=plt.cm.coolwarm, transform=ccrs.PlateCarree(), cbar_kwargs={"shrink": 0.6}
)

plt.title("Land Surface Temperature Around the Carribean")

Kunnen we aan de hand van de bovenstaande kaart de temperatuurverandering door te tijd heen zien? Waarom wel/niet?

We gaan nu kijken naar Bonaire specifiek om te temperatuurverandering door de tijd heen te zien. Zoek de latitude en longitude op op google maps en vul die hieronder in:

In [ ]:
latitude_bonaire = YOUR_CODE_HERE
longitude_bonaire = YOUR_CODE_HERE

Met deze data kunnen we nu een grafiek maken van de temperatuuur vanaf 1951 tot en met 2022.

In [ ]:
# Step 1: Mask out NaN values
valid_data = ds.stl1.where(ds.stl1.notnull(), drop=True)

# Step 2: Find the nearest valid point
bonaire = valid_data.sel(latitude=latitude_bonaire, longitude=longitude_bonaire, method="nearest")

plt.figure(figsize=(10, 6))
bonaire.plot(label="Temperatuurdata")
plt.xlabel("Jaar")
plt.ylabel("Temperatuur in Celsius")
plt.title("Temperatuurverandering in Bonaire")
plt.legend()
plt.show()

Hier kunnen we al een beetje een stijging zien in de temperatuur, maar we kunnen een trendlijn toevoegen om dit beter te zien.

In [ ]:
# Convert time to numerical values for regression
time_numeric = (ds.time - ds.time[0]) / np.timedelta64(1, 'D')  # Convert time to days since start

# Calculate the trend line (linear regression)
coefficients = np.polyfit(time_numeric, bonaire, deg=1)  # deg=1 for linear trend
trend_line = np.polyval(coefficients, time_numeric)

# Add the trend line to the dataset
bonaire["trend"] = (("time"), trend_line)

plt.figure(figsize=(10, 6))
bonaire.plot(label="Temperatuurdata")
bonaire.trend.plot(label="Trendlijn", color="red", linewidth=2)
plt.xlabel("Jaar")
plt.ylabel("Temperatuur in Celsius")
plt.title("Temperatuurverandering in Bonaire met trendlijn")
plt.legend()
plt.show()

De trendlijn laat de stijging wat duidelijker zien. Wel zien we dat de lijn heel ver boven de trendlijn gaat, en dan weer heel ver onder. Laten we inzoomen op een jaar om dit beter te zien. Je mag zelf kiezen welk jaar en dan kan je het hieronder invullen. Als je het leuk vind kun je ook meerdere jaartallen uitproberen!

In [ ]:
jaar = YOUR_CODE_HERE

# Step 2: Find the nearest valid point
time_sliced = bonaire.sel(time=slice(f"{jaar}-01-01", f"{jaar}-12-31"))

time_sliced.plot()
plt.title(f"Temperatuurverandering in {jaar}")
plt.xlabel("Maand")
plt.ylabel("Temperatuur in Celsius")


Nu dat je hebt ingezoomd op de individuele jaren, verklaar waarom de temperatuur zoveel uitwijkt van de trendlijn